In [ ]:
from random import *
from Crypto.Util.number import getPrime, isPrime
from math import gcd
from math import log2

### Question 1.– 
Implanter une fonction xgcd(p, q) qui retourne les coefficients de Bezout associés à deux
entiers p et q (autrement dit, des entiers u et v tels que up + vq = pgcd(p, q)).
On rappelle que pour p ≡ 3 mod 4, les racines carrées d’un carré x ∈ Z/pZ sont ±x
(p+1)/4. On rappelle
également que l’on peut déduire les racines carrées modulo n = pq de celles modulo p et q au moyen de
l’isomorphisme des restes chinois.

In [ ]:
def xgcd(a,b):
  x,y=1,0
  u,v=0,1
  while b != 0:
   q,r=a//b,a%b
   m,n=x-u*q,y-v*q #On fait l'opération L1-q*L2 (1)
   a,b=b,r #On fait un décalage car la technique le veut. Le quotient devant le numérateur et le reste devient le quotient 
   x,y=u,v #Maintenant comme on fait L2-L3 les coefficients ne sont plus les mêmes pour la ligne de calcul(1), 
   u,v=m,n #donc on va echanger.
  reste=a
  return [reste,x,y] #a est le reste b 

### Question 2.– 
Implanter une fonction square_roots(x, p, q, n, u, v) qui retourne la liste des racines
carrées de x modulo n, où n = pq, avec p et q deux nombres premiers distincts congrus à 3 modulo 4, et
où u et v sont les coefficients de Bezout associés à p et q. On supposera que x est bien un carré modulo n.

In [ ]:
def square_roots(x, p, q, n, u, v):
    if p % 4 == 3 and q % 4 == 3:
        RacineCarre1 = (p + 1) // 4  #=1
        RacineCarre2 = (q + 1) // 4  #=1 Je sais pas si je devais quand même les ecrire ou pas 
        a = pow(x, RacineCarre1, p)
        b = pow(x, RacineCarre2, q)
        c1 = (a * v * q + b * u * p) % n # formule finale du thoereme des restes chinois. v= inverse modulaire de q modulo p et u inverse modulaire de p modulo q
        c2 = (-a * v * q + b * u * p) % n #racine 2 
        c3 = (a * v * q - b * u * p) % n #racine 3
        c4 = (-a * v * q - b * u * p) % n #racine 4
        return [c1, c2, c3, c4]
    else:
        print("Erreur: p et q ne sont pas congrus à 3 modulo 4.")
        return []  # Retourne une liste vide

### Question 3.– 
Implanter une fonction keygen_rabin(t) qui engendre une paire de clés pour le cryptosystème de Rabin, dont le module n est un entier de taille au moins t bits.

In [ ]:


def keygen_rabin(t):   # On genere les clés
   Taille_p_q=t//2 #ici ce que je veux faire c'est initialiser des p et des q qui sont des nombres de taille  inferieur a celle de t tel que taille(p*q)=taille(n).
   Taille_t=t//1 # La je définis qu'elle est la taille de t en bits
   p=getPrime(Taille_p_q) #ici je veux des nombres premier de taille p et q 
   q=getPrime(Taille_p_q) 
   n=p*q                  
   Taille_n=(int(log2(n))//1)+1 #Taille de n en bits
   while p%4!=3 or q%4!=3: #Si tous ce passe bien comme on le voudrait alors on rentre pas dans la boucle. 
        p=getPrime(Taille_p_q) 
        q=getPrime(Taille_p_q)
        n=p*q
    #while int(log2(n)//1)+1<Taille_p_q+1:, n=n*2
   u,v=xgcd(p,q)[1],xgcd(p,q)[2] #Je lance ma fonction qui calcule les coefficients de bezout de p et q.
   return [n,p,q,u,v] 

### Question 4.– 
Implanter les fonctions de chiffrement encrypt_rabin_1(plaintext, pk) et de déchiffrement decrypt_rabin_1(ciphertext, sk) du cryptosystème initial de Rabin. On stockera dans les
variables pk et sk les clés publique et privée, et dans les variables plaintext et ciphertext le clair et le
chiffré.

In [ ]:
def encrypt_rabin_1(plaintext, pk):  #n=pk clé publique et plainText message clair
      c=pow(plaintext,2,pk)
      return c

In [ ]:
def decrypt_rabin_1(ciphertext, sk):
   p, q, u, v = sk[1:] #On fait ça pour mieux utiliser square_roots
   roots = square_roots(ciphertext, p, q, p * q, u, v)
   if roots is not None:
      return roots
   else:
      return print("Erreur: Impossible de trouver les racines carrées.")

### Question 5.– 
Écrire une procédure de test TEST1(t) qui s’assure, sur plusieurs exemples, que votre
implantation est valide pour la taille de module t. On vérifiera donc que le message d’origine plaintext
figure dans la liste retournée par decrypt_rabin_1(ciphertext, sk).

In [ ]:
def TEST1(t):
    c=0
    m=randint(0, 100)
    keygen_rabin(t)
    Cle_Secrete = keygen_rabin(t)  # Stockez la nouvelle clé secrète générée dans Cle_Secrete
    pk = Cle_Secrete[0]      #On dit que pk est n
    c = encrypt_rabin_1(m, pk)
    Message_eventuel = decrypt_rabin_1(c, Cle_Secrete) #On cherche le mot eventuelle c'est à dire qu'on regarde une liste ou notre mot devrait normalmeent se trouver
    Testeur = 0
    for i in range(0, len(Message_eventuel)): #La on teste seulement l'existence et l'unicité du mot m dans notre liste.
        if Message_eventuel[i] == m:
            Testeur += 1
            
    if Testeur != 1:
        return print("Erreur")
    else:
        return print("Succès") # Ça montre qu'il est unique et qu'il existe bien.

In [ ]:
def encrypt_rabin2(m, pk, padding):
      message_padding=m*pow(2,padding,pk)
      c=encrypt_rabin_1(message_padding,pk)  # 10 = 2^1 on passe de 100 car 2*2^3=16 et 16 s'ecrit en binaire 10000
      return c 


### Question 6.– 
Implanter les fonctions de chiffrement encrypt_rabin_2(m, pk, padding) et de déchiffrement decrypt_rabin_2(c, sk, padding) du cryptosystème de Rabin avec padding.

In [ ]:
def decrypt_rabin2(c, sk, padding):
    Liste_message_eventuel = decrypt_rabin_1(c, sk) 
    s=0
    for i in range(0,len(Liste_message_eventuel)): #on parcourt notre Liste de message eventuel
        if Liste_message_eventuel[i]%pow(2,padding)== 0: #On detecte la position de où se trouve celui qui est divisible par 2^padding.
            L=Liste_message_eventuel[i] // pow(2,padding) #On veut le quotient
            s=s+1
    if s!=1:
        return print(s)
    else:
        return L

### Question 7.– 
Écrire une procédure de test TEST2(t) qui teste lorsque le déchiffrement de Rabin avec
padding est correct. En pratique, on prendra comme padding la valeur ⌊t/2⌋, où t est le paramètre qui a
été choisi pour engendrer les clés.
Pour de petites valeurs de t (disons, inférieures à 10), observez-vous parfois des erreurs de déchiffrement ?
Si oui, commentez.

In [ ]:
def TEST2(t):  
      padding=t//2  #On veut le quotient par 2 pour avoir la partie inferieur divisée par 2
      m=randint(1,1024) 
      Cle_Secrete=keygen_rabin(t) #Notre clé secrete = toute la liste
      Cle_Publique=Cle_Secrete[0] #Notre clé publique = n qui est le premier element qu'affiche keygen_rabin
      message_padding=m*pow(2,padding,Cle_Publique)
      if message_padding> Cle_Publique :
          return print("Rentrer t plus petit")
      else :
          c=encrypt_rabin2(m,Cle_Publique,padding)
          Message_eventuel=decrypt_rabin2(c,Cle_Secrete,padding)
          if m==Message_eventuel: #Si ça fonctionne alors le quotient va nous donner m
              return print("Succès") 
          else :
              return print("Echec")